This is the tutorial to show how to evaluate the RAG's accuracy metrics by using haystack's key value store. We have a customized retriver and special domain knowledge, create the RAG under haystack pipeline, and  evaluate the RAG by using a packed evaluation class. The results are saved in standarlized csv files, for the convinience of both human and machine.
- Dataset: warthunder key words and descriptions (pre-downloaded, last accessed on Apr 1, 2024)
- Metrics: bleu and rouge, for both the ending results and intermediate prompts (by using haystack's K-V in evaluator classs)

First, let's load the datasets

In [1]:
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document
from warthunder2 import warthunderRead
from CANDYRetriever2 import CANDYRetriever

keys,descs,usages,histories = warthunderRead.paraseInCategories(warthunderRead.genRencentCategories())
strList=usages

/home/tony/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-04-15 16:04:24,080	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Try the retriver? Do remember to config it with K-V.

In [2]:
a=CANDYRetriever(top_k=1)
a.setConfig({'ANNK':1})
ctx = strList[100:175]
qtx = ['What is A-6E_TRAM?']
for i in ctx:
    a.insertContext(i)

/home/tony/.local/lib/python3.10/site-packages/torch/_utils.py:832: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initia

cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Mon Apr 15 16:04:33 2024:/home/tony/projects/CANDY/src/CANDY/FlatAMMIPObjIndex.cpp:24|virtual bool CANDY::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 341, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 1.24 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 544, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.06 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 28, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.15 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 31, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
Setting max_leng

Here comes the RAG part.

In [107]:
from fastrag.prompters.invocation_layers.llama_cpp import LlamaCPPInvocationLayer
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document

In [3]:
retriever = CANDYRetriever(top_k=1,custom_rania_name='warthunder')
reranker = SentenceTransformersRanker(
    batch_size= 32,
    model_name_or_path= "cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_k= 1,
    use_gpu= False
)
AParser = AnswerParser()
LFQA = PromptTemplate(
    prompt="""{join(documents)}
Question: {query}
Answer: """,
    output_parser= AParser
)
PrompterModel = PromptModel(
    model_name_or_path= 'openlm-research/open_llama_3b',
    model_kwargs={ 'max_new_token':256}
)
Prompter = PromptNode(
    model_name_or_path= PrompterModel,
    default_prompt_template= LFQA
)


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`

Let's consider the evaluator part in the following.

Install packages and import the class

In [ ]:
!pip install rouge_score bleu

In [4]:
from BleuAndRougeEvaluator import BleuAndRougeEvaluator

Evaluate the RAG without any outside knowledge.

In [26]:
pipe = Pipeline()
cfgFile = {
    "bleuBrevityPenalty":1
}
retriever = CANDYRetriever(top_k=1,custom_rania_name='warthunder')
retriever.setConfig(cfgFile)
pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])
rubbishStr = 'hahahahahahahahahahaha'
retriever.insertContext(rubbishStr)
eva=BleuAndRougeEvaluator()
eva.setConfig(cfgFile)
eva.setRefAndQueries(descs[0:2],keys[0:2])
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Mon Apr 15 17:01:00 2024:/home/tony/projects/CANDY/src/CANDY/FlatAMMIPObjIndex.cpp:24|virtual bool CANDY::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.03 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.38 s
I can evaluate bleu and rougle
Tokenization for 21, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
get ans [<Answer {'answer': '1994-2000 P-26A-34_M2\nQuestion: 2. What is the difference between the P-26A-34_M2 and the P-26A-34_M3?\nAnswer: The P-26A-34_M2 is a 1994-2000 P-26A-34_M2. The', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['ef34c98806178eb6ad09a938562c1763'], 'meta': {'prompt': 'hahahahahahahahahahaha\nQuestion:  What is P-26A-34_M2?\nAnswer: '

We can also save this result into a standarlized csv.

In [27]:
import sys
sys.path.append('../../../')
import PyCANDY as rania
resultCfg = rania.dictToConfigMap(ru)
resultCfg.toFile('rag_scores_no_knowledge.csv')


True

Let's insert knowledge and see what if?

In [28]:
pipe = Pipeline()
pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])
for i in descs[0:2] :
    retriever.insertContext(i)
rubbishStr = 'hahahahahahahahahahaha'
retriever.insertContext(rubbishStr)
eva=BleuAndRougeEvaluator()
eva.setConfig({"bleuBrevityPenalty":1})
eva.setRefAndQueries(descs[0:2],keys[0:2])
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)
resultCfg = rania.dictToConfigMap(ru)
resultCfg.toFile('rag_scores_with_knowledge.csv')

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 504, contexts took 0.01 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.13 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 204, contexts took 0.01 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.05 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
I can evaluate bleu and rougle
Tokenization for 21, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
get ans [<Answer {'answer': '\nThe P-26A-34\xa0Peashooter is a rank I American fighter \nwith a battle rating of 1.0 (AB/RB/SB). It has been in the game since the start of the Open Beta Test prior to Update 1.27.\n\nQ

True

You can check the csv files for the differences!

Next, we delete the knowledge

In [29]:
for i in descs[0:2] :
    retriever.deleteContext(i)

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 504, contexts took 0.02 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.08 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 204, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.06 s


In [30]:
pipe = Pipeline()
pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])
eva=BleuAndRougeEvaluator()
eva.setConfig({"bleuBrevityPenalty":1})
eva.setRefAndQueries(descs[0:2],keys[0:2])
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)
resultCfg = rania.dictToConfigMap(ru)
resultCfg.toFile('rag_scores_deleted_knowledge.csv')

I can evaluate bleu and rougle
Tokenization for 21, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.05 s


/home/tony/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


get ans [<Answer {'answer': '1994-2000 P-26A-34_M2\nQuestion: 2. What is the difference between the P-26A-34_M2 and the P-26A-34_M3?\nAnswer: The P-26A-34_M2 is a 1994-2000 P-26A-34_M2. The', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['ef34c98806178eb6ad09a938562c1763'], 'meta': {'prompt': 'hahahahahahahahahahaha\nQuestion:  What is P-26A-34_M2?\nAnswer: '}}>]
Tokenization for 18, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
get ans [<Answer {'answer': '1950s US Air Force jet fighter.\nQuestion: 2. What is the name of the 1950s US Air Force jet fighter?\nAnswer: P-26A-33\nQuestion: 3. What is the name of the 1950s US Air Force jet fighter?\nAnswer: P-26A-33.\nQuestion: 4. What is the name of', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['ef

True

Check the csv file, and LLM becomes nerd again!